# Support Vector Machine (SVM)

## Importing the libraries

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Importing the dataset

In [11]:
df = pd.read_csv("./Dataset/riceClassification.csv")

X=df.iloc[:,1:-1]
y=df.iloc[:,-1]

## Splitting the dataset into the Training set and Test set

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Training the SVM model on the Training set

In [13]:
svm_clf = Pipeline([("scaler", StandardScaler()),("svc", SVC(kernel="rbf", C=10.0, gamma="scale", random_state=42))])
svm_clf.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('svc', SVC(C=10.0, random_state=42))])

## Predicting the Test set results

In [14]:
y_pred = svm_clf.predict(X_test)

# Report
print("\n📊 SVM Classification Report (All Features):")
print(classification_report(y_test, y_pred))


📊 SVM Classification Report (All Features):
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1640
           1       0.99      1.00      0.99      1997

    accuracy                           0.99      3637
   macro avg       0.99      0.99      0.99      3637
weighted avg       0.99      0.99      0.99      3637



## Making the Confusion Matrix

In [15]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Confusion Matrix:
 [[1620   20]
 [   8 1989]]
Accuracy: 99.23%


# ADMM Top 5 features(SVM)

## ADMM Feature Selection

In [16]:
def select_top_k_via_admm(X, y, lam=0.01, rho=1.0, iters=200, k=5):
    Xn = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-12)
    Xn = Xn.values if hasattr(Xn, "values") else np.asarray(Xn)
    yv = y.values if hasattr(y, "values") else np.asarray(y)

    n, p = Xn.shape
    w = np.zeros(p)
    z = np.zeros(p)
    u = np.zeros(p)

    XtX = Xn.T @ Xn
    Xty = Xn.T @ yv
    I = np.eye(p)

    def soft_threshold(v, t):
        return np.sign(v) * np.maximum(np.abs(v) - t, 0.0)

    for _ in range(iters):
        w = np.linalg.solve(XtX + rho * I, Xty + rho * (z - u))
        z = soft_threshold(w + u, lam / rho)
        u = u + (w - z)

    idx = np.argsort(-np.abs(z))[:k]
    selected_cols = list(X.columns[idx])
    return selected_cols

top5_cols = select_top_k_via_admm(X, y, lam=0.01, rho=1.0, iters=300, k=5)
print("Selected (Top-5 ADMM):", top5_cols)

X = X[top5_cols]


Selected (Top-5 ADMM): ['Area', 'EquivDiameter', 'MajorAxisLength', 'ConvexArea', 'AspectRation']


## Splitting the dataset into the Training set and Test set

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Training the SVM model on the Training set

In [18]:
svm_clf = Pipeline([("scaler", StandardScaler()),("svc", SVC(kernel="rbf", C=10.0, gamma="scale", random_state=42))])
svm_clf.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svc', SVC(C=10.0, random_state=42))])

## Predicting the Test set results

In [20]:
y_pred = svm_clf.predict(X_test)

# Report
print("\n📊 SVM Classification Report (Top-5 ADMM Features):")
print(classification_report(y_test, y_pred))


📊 SVM Classification Report (Top-5 ADMM Features):
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1640
           1       0.99      0.99      0.99      1997

    accuracy                           0.99      3637
   macro avg       0.99      0.99      0.99      3637
weighted avg       0.99      0.99      0.99      3637



## Making the Confusion Matrix

In [21]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Confusion Matrix:
 [[1619   21]
 [  11 1986]]
Accuracy: 99.12%
